In [1]:
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from scikeras.wrappers import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

from sklearn.compose import ColumnTransformer

2022-10-01 13:20:33.752890: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-01 13:20:33.948382: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-10-01 13:20:34.614449: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-10-01 13:20:34.614570: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

In [2]:
# load dataset
property_df = pd.read_csv("../data/curated/full_data.csv")
X = property_df
X2 = X[['SA2_NAME21']].groupby(by="SA2_NAME21")["SA2_NAME21"].count()
X2 = pd.DataFrame(X2)
X2.columns=["count"]
single_counts = X2[X2['count']==1].reset_index()['SA2_NAME21'].tolist() # can improve this later with bootstrapping
X = X[~X["SA2_NAME21"].isin(single_counts)]
X.head(1)



,index,address,cost_text,Bed,Bath,Park,property_type,desc_head,LocID,longitude,latitude,Median_tot_prsnl_inc_weekly,Median_rent_weekly,Median_tot_fam_inc_weekly,Median_tot_hhd_inc_weekly,total_population,SA2_NAME21,duration_mins,Offence Count,SA2_CODE_2021
0,https://www.domain.com.au/warrandyte-vic-3113-...,Warrandyte VIC 3113,875.0,5,3,3,House,mud brick magic,211021262,145.209992,-37.747366,930.0,481.0,3051.0,2779.0,8781.0,Warrandyte - Wonga Park,>50.0,859.0,211021262.0


In [3]:
import geopandas as gpd
# sf stands for shape file
australia_sf = gpd.read_file("../data/raw/shapefiles/Statistical_area_level2/SA2_2021_AUST_GDA2020.shp")
australia_sf.head(3)
# seperate suburbs that are in Victoria
vic_sf = australia_sf[australia_sf['STE_NAME21'] == 'Victoria']

# drop the null location ID values
vic_sf.dropna(inplace=True)
vic_sf.head(1)
vic_sf.where(vic_sf["SA2_NAME21"] == "Seaford (Vic.)").dropna()

/tmp/ipykernel_58905/1187906559.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vic_sf.dropna(inplace=True)


,SA2_CODE21,SA2_NAME21,CHG_FLAG21,CHG_LBL21,SA3_CODE21,SA3_NAME21,SA4_CODE21,SA4_NAME21,GCC_CODE21,GCC_NAME21,STE_CODE21,STE_NAME21,AUS_CODE21,AUS_NAME21,AREASQKM21,LOCI_URI21,geometry
1104,214011375,Seaford (Vic.),0,No change,21401,Frankston,214,Mornington Peninsula,2GMEL,Greater Melbourne,2,Victoria,AUS,Australia,12.4321,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"POLYGON ((145.12445 -38.10103, 145.12447 -38.1..."


In [4]:
vic_centroid_sf = vic_sf
vic_centroid_sf['geometry'] = vic_sf['geometry'].to_crs('+proj=cea').centroid.to_crs(vic_sf['geometry'].crs)
cbd_loc = vic_centroid_sf.where(vic_centroid_sf['SA2_NAME21'] == 'Melbourne CBD - North').dropna()['geometry']
cbd_loc

/home/sesshah/anaconda3/lib/python3.9/site-packages/geopandas/geodataframe.py:1472: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


780    POINT (144.96089 -37.80918)
Name: geometry, dtype: geometry

In [5]:
# function taken from https://stackoverflow.com/a/56661833

from math import radians, cos, sin, asin, sqrt, inf

def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles
    return c * r

# this function converts two floats to a point that is recognised by Shapely
def point_to_dist(row):
    dist = haversine(row["geometry"].y, row["geometry"].x, cbd_loc.y, cbd_loc.x)
    return dist


In [6]:
X2 = X[['SA2_NAME21']].groupby(by="SA2_NAME21")["SA2_NAME21"].count()
X2 = pd.DataFrame(X2)
X2.columns=["count"]
X3 = vic_centroid_sf.join(X2, how='inner',on='SA2_NAME21')

X3['cbd_dist'] = X3.apply (lambda row: point_to_dist(row), axis=1)
X3.sort_values(by=['cbd_dist'], inplace=True)
X3

,SA2_CODE21,SA2_NAME21,CHG_FLAG21,CHG_LBL21,SA3_CODE21,SA3_NAME21,SA4_CODE21,SA4_NAME21,GCC_CODE21,GCC_NAME21,STE_CODE21,STE_NAME21,AUS_CODE21,AUS_NAME21,AREASQKM21,LOCI_URI21,geometry,count,cbd_dist
780,206041504,Melbourne CBD - North,1,New,20604,Melbourne City,206,Melbourne - Inner,2GMEL,Greater Melbourne,2,Victoria,AUS,Australia,0.5616,http://linked.data.gov.au/dataset/asgsed3/SA2/...,POINT (144.96089 -37.80918),252,0.000000
781,206041505,Melbourne CBD - West,1,New,20604,Melbourne City,206,Melbourne - Inner,2GMEL,Greater Melbourne,2,Victoria,AUS,Australia,1.0051,http://linked.data.gov.au/dataset/asgsed3/SA2/...,POINT (144.95871 -37.81672),200,0.728392
779,206041503,Melbourne CBD - East,1,New,20604,Melbourne City,206,Melbourne - Inner,2GMEL,Greater Melbourne,2,Victoria,AUS,Australia,0.8020,http://linked.data.gov.au/dataset/asgsed3/SA2/...,POINT (144.96860 -37.81277),138,0.917602
771,206041117,Carlton,0,No change,20604,Melbourne City,206,Melbourne - Inner,2GMEL,Greater Melbourne,2,Victoria,AUS,Australia,1.8187,http://linked.data.gov.au/dataset/asgsed3/SA2/...,POINT (144.96781 -37.80049),218,1.103787
786,206041510,West Melbourne - Residential,1,New,20604,Melbourne City,206,Melbourne - Inner,2GMEL,Greater Melbourne,2,Victoria,AUS,Australia,0.8247,http://linked.data.gov.au/dataset/asgsed3/SA2/...,POINT (144.94977 -37.80857),67,1.237667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1009,212031557,Cranbourne East - North,1,New,21203,Casey - South,212,Melbourne - South East,2GMEL,Greater Melbourne,2,Victoria,AUS,Australia,11.0254,http://linked.data.gov.au/dataset/asgsed3/SA2/...,POINT (145.31291 -38.10222),12,47.402490
990,212021293,Berwick - North,0,No change,21202,Casey - North,212,Melbourne - South East,2GMEL,Greater Melbourne,2,Victoria,AUS,Australia,19.7480,http://linked.data.gov.au/dataset/asgsed3/SA2/...,POINT (145.34885 -38.02511),32,47.426478
1010,212031558,Cranbourne East - South,1,New,21203,Casey - South,212,Melbourne - South East,2GMEL,Greater Melbourne,2,Victoria,AUS,Australia,5.3272,http://linked.data.gov.au/dataset/asgsed3/SA2/...,POINT (145.30423 -38.12247),13,47.691723
973,211051278,Lilydale - Coldstream,0,No change,21105,Yarra Ranges,211,Melbourne - Outer East,2GMEL,Greater Melbourne,2,Victoria,AUS,Australia,109.3817,http://linked.data.gov.au/dataset/asgsed3/SA2/...,POINT (145.38884 -37.71375),9,48.377052


In [7]:
pruned_X3 = X3[['SA2_NAME21', "cbd_dist"]]
pruned_X3.set_index('SA2_NAME21', inplace=True)
pruned_X3
#pruned_X3.join(X, how='left', on="SA2_NAME21")

,cbd_dist
SA2_NAME21,
Melbourne CBD - North,0.000000
Melbourne CBD - West,0.728392
Melbourne CBD - East,0.917602
Carlton,1.103787
West Melbourne - Residential,1.237667
...,...
Cranbourne East - North,47.402490
Berwick - North,47.426478
Cranbourne East - South,47.691723


In [8]:
X_dists = X.join(pruned_X3, how='left', on="SA2_NAME21")

In [9]:
sl = 50
scaler = StandardScaler()
numeric_features = [ 
                  "Median_tot_prsnl_inc_weekly", 
                  "Median_rent_weekly",
                  "Median_tot_fam_inc_weekly", 
                  "Median_tot_hhd_inc_weekly", 
                  "total_population",
                  "Bed", 
                  "Bath", 
                  "Park",
                  "Offence Count",
                  "cbd_dist"]
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', scaler, numeric_features)])

y = X_dists['cost_text'].tolist()
print("\tnum training instances: ", len(y))

	num training instances:  10860


In [10]:
data = X_dists.drop(["index", "address", "desc_head", "latitude", "longitude", "LocID", "SA2_CODE_2021", 
              "cost_text", "SA2_NAME21"], axis=1)
data.astype({ 
          "Median_tot_prsnl_inc_weekly":"float", 
          "Median_rent_weekly":"float",
          "Median_tot_fam_inc_weekly":"float", 
          "Median_tot_hhd_inc_weekly":"float", 
          "total_population": "int64",
          "duration_mins": "category", 
          "Bed":"int32", 
          "Bath":"int32", 
          "Park":"int32",
          "property_type":"category",
          #"SA2_NAME21": "category",
          "Offence Count": "int32",
          "cbd_dist": "float"}).dtypes
data.shape

(10860, 12)

In [11]:
# define base model
def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(100, input_shape=(12,), kernel_initializer='normal', activation='relu'))
    model.add(Dense(32, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal', activation="sigmoid"))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [12]:
estimator = KerasRegressor(model=baseline_model, epochs=200, verbose=0)
kfold = KFold(n_splits=10, shuffle=True, random_state=30027)

results = cross_val_score(estimator, preprocessor.fit_transform(data), y, cv=kfold)
print(results)

2022-10-01 13:20:56.052124: E tensorflow/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: CUDA_ERROR_SYSTEM_DRIVER_MISMATCH: system has unsupported display driver / cuda driver combination
2022-10-01 13:20:56.052168: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: uwu
2022-10-01 13:20:56.052176: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: uwu
2022-10-01 13:20:56.052288: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 515.65.1
2022-10-01 13:20:56.052312: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 390.154.0
2022-10-01 13:20:56.052318: E tensorflow/stream_executor/cuda/cuda_diagnostics.cc:313] kernel version 390.154.0 does not match DSO version 515.65.1 -- cannot find working devices in this configuration
2022-10-01 13:20:56.052616: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary 

[nan nan nan nan nan nan nan nan nan nan]


/home/sesshah/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
10 fits failed out of a total of 10.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1 fits failed with the following error:
Traceback (most recent call last):
  File "/home/sesshah/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/sesshah/anaconda3/lib/python3.9/site-packages/scikeras/wrappers.py", line 762, in fit
    self._fit(
  File "/home/sesshah/anaconda3/lib/python3.9/site-packages/scikeras/wrappers.py", line 931, in _fit
    self._fit_keras_model(
  File "/home/sesshah/anaconda3/lib/python3.9/site